In [8]:
from PIL import Image 
import requests 
from transformers import AutoModelForCausalLM ,BitsAndBytesConfig
from transformers import AutoProcessor 

model_id = "microsoft/Phi-3-vision-128k-instruct" 
quant_config = BitsAndBytesConfig(
        load_in_4bit=True
        
    )
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", 
                                            trust_remote_code=True, torch_dtype="auto", 
                                            _attn_implementation='flash_attention_2' ,
                                            quantization_config =quant_config) #use _attn_implementation='eager' to disable flash attention



Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.04s/it]


In [2]:
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
gc.collect()

581

In [11]:
import torch
torch.cuda.empty_cache()

In [12]:
import gc
import torch
del model
gc.collect()
torch.cuda.empty_cache() 

In [6]:
gc.collect()

0

In [28]:
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in the image?"}, 
] 


image = Image.open('../images/image.png')


In [29]:
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 


In [30]:

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)

c:\Users\jvish\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


The image shows a person riding a bicycle on a road with a field of yellow flowers in the background. There is a red car parked on the side of the road and a person walking in the distance. The sky is clear and blue.


In [31]:
import torch
torch.cuda.empty_cache()


In [32]:
#ask a follow up quesiton from same image
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": response}, 
    {"role": "user", "content": "What is the meaning of the image?"},
]

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt,return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

def stream_response(model, inputs, generation_args, processor):
    generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args, return_dict_in_generate=True, output_scores=True)
    sequences = generate_ids.sequences[:, inputs['input_ids'].shape[1]:]
    for token_id in sequences[0]:
        token = processor.batch_decode(token_id.unsqueeze(0), skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        yield token



In [33]:
#ask a follow up quesiton from same image
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": response}, 
    {"role": "user", "content": "What is the meaning of the image?"},
]

image = Image.open('../images/image.png')

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)

The image conveys a sense of freedom, adventure, and the joy of outdoor activities. The person riding the bicycle appears to be enjoying the ride, and the open field and clear sky suggest a pleasant day for such activities.


In [34]:
import torch    
torch.cuda.empty_cache()

In [35]:
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": response},
        {"role": "user", "content": "What is shown in this image?"}, 
    {"role": "assistant", "content": response}, 
    {"role": "user", "content": "What is color of flowers?"},
] 

image = Image.open('../images/image.png')

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

print("generating")
# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)

generating
The flowers in the field are yellow.


In [38]:
import torch
torch.cuda.empty_cache()

In [39]:
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": response},
    {"role": "user", "content": "What is shown in this image?"}, 
    {"role": "assistant", "content": response}, 
    {"role": "user", "content": "What is color of flowers?"},
    {"role": "assistant", "content": "The flowers in the field are yellow."},
    {"role": "user", "content": "What is the color of the sky?"}
] 

image = Image.open('../images/image.png')

prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 

print("generating")
# remove input tokens 
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 

print(response)

c:\Users\jvish\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\generation\configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


generating
The sky is blue.


In [1]:
from PIL import Image 
import requests 
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

model_id = "microsoft/Phi-3-vision-128k-instruct" 

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='flash_attention_2') # use _attn_implementation='eager' to disable flash attention

processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 

messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}, 
    {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."}, 
    {"role": "user", "content": "Provide insightful questions to spark discussion."} 
] 


c:\Users\jvish\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\jvish\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\jvish\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jvish\.cache\huggingface\hub\models--microsoft--Phi-3-vision-128k-instruct. Caching files will still work but in a degraded version that

In [24]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\jvish\appdata\local\programs\python\python312\lib\site-packages\flash_attn-2.5.9.post1-py3.12-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [11]:
messages = []

if messages == []:
    print("no image found in messages")
else:
    print("image found in messages")
    

no image found in messages


In [1]:
! pip install chromadb sentence-transformers

DEPRECATION: Loading egg at c:\users\jvish\appdata\local\programs\python\python312\lib\site-packages\flash_attn-2.5.9.post1-py3.12-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [7]:
from sentence_transformers import SentenceTransformer
import chromadb
import numpy as np

# Initialize conversation history at different levels
lvl1_history = []
lvl2_history = []
lvl3_history = []

class ConversationManager:
    def __init__(self, level_threshold=5):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.client = chromadb.Client()
        self.collection = self.client.create_collection(name="conversational2_embeddings")
        self.messages = []
        self.level_threshold = level_threshold
        self.current_level = 1

    def add_message(self, message):
        embedding = self.model.encode(message).tolist()  # Convert numpy array to list for ChromaDB
        self.collection.add(embeddings=[embedding], metadatas=[{"message": message}], ids=[str(len(self.messages))])
        self.messages.append(message)
        self.update_level()

    def update_level(self):
        if len(self.messages) > self.level_threshold:
            self.current_level += 1
            self.level_threshold *= 2  # Adjust the threshold multiplier as needed

    def print_messages_by_level(self):
        # Simulate distributing messages into levels based on current_level and thresholds
        levels = {i: [] for i in range(1, self.current_level + 1)}
        current_threshold = 5
        current_level = 1
        
        for i, message in enumerate(self.messages):
            levels[current_level].append(message)
            if len(levels[current_level]) >= current_threshold:
                current_level += 1
                current_threshold *= 2  # Adjust the threshold multiplier as needed

        for level, msgs in levels.items():
            print(f"Level {level}:")
            for msg in msgs:
                print(f" - {msg}")

# Initialize the conversation manager
conversation_manager = ConversationManager(level_threshold=5)

# Simulating adding messages to ChromaDB
messages = [
    "Hello, how can I help you today?",
    "I have an issue with my order.",
    "Can you provide more details about your issue?",
    "My order arrived damaged.",
    "I'm sorry to hear that. We'll send a replacement right away.",
    "Thank you for your help.",
    "How long will it take to arrive?",
    "It should arrive in 3-5 business days.",
    "Can I track my order?",
    "Yes, you will receive a tracking number shortly.",
]

# Add messages to ChromaDB
for msg in messages:
    conversation_manager.add_message(msg)

# Print messages grouped by levels
conversation_manager.print_messages_by_level()


c:\Users\jvish\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Level 1:
 - Hello, how can I help you today?
 - I have an issue with my order.
 - Can you provide more details about your issue?
 - My order arrived damaged.
 - I'm sorry to hear that. We'll send a replacement right away.
Level 2:
 - Thank you for your help.
 - How long will it take to arrive?
 - It should arrive in 3-5 business days.
 - Can I track my order?
 - Yes, you will receive a tracking number shortly.
